In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def compute_wins(tensor):
    zeros = torch.zeros_like(tensor)
    argmax = tensor.argmax(1).reshape(-1, 1)
    zeros.scatter_(1, argmax, 1)
    wins = zeros.sum(dim=0)
    return wins

def string_to_tensor(tensor_string, softmax=False):
    # Extracts tensor values from string and returns a PyTorch tensor
    tensor_data = eval(tensor_string.split('tensor(')[1].split(', device')[0])
    t = torch.tensor(tensor_data)
    softmax_t = F.softmax(t, dim=1)
    wins = compute_wins(softmax_t)

    return (softmax_t if softmax else t, wins)

def extract_last_two_tensors(file_path, softmax=True):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
    except Exception as e:
        return str(e), None, None

    last_delimiter_index = content.rfind('<<<--->>>')
    if last_delimiter_index == -1:
        return f"No delimiter found in {file_path}", None, None

    last_section = content[last_delimiter_index + len('<<<--->>>'):].strip()
    lines = last_section.splitlines()
    tensors = []
    tensor_data = []
    capturing_tensor = False

    for line in lines:
        if 'tensor([' in line:
            capturing_tensor = True
            tensor_data.append(line.strip())
        elif capturing_tensor:
            tensor_data.append(line.strip())
            if 'requires_grad=True' in line:
                tensor_string = '\n'.join(tensor_data)
                tensor_obj = string_to_tensor(tensor_string, softmax=softmax)
                tensors.append(tensor_obj)
                tensor_data = []
                capturing_tensor = False

    if len(tensors) >= 2:
        return None, tensors[-2], tensors[-1]
    elif len(tensors) == 1:
        return f"Only one tensor found in {file_path}", tensors[-1], None
    else:
        return f"No tensors found after the delimiter in {file_path}", None, None

def process_directory(directory_path):
    tensor_outputs = {}
    for filename in os.listdir(directory_path):
        if filename.endswith(".log"):  # Adjust the extension based on your files
            file_path = os.path.join(directory_path, filename)
            error, tensor1, tensor2 = extract_last_two_tensors(file_path)
            tensor_outputs[filename] = {"error": error, "normal": tensor1, "reduction": tensor2}
    return tensor_outputs

In [3]:
all_tensors = process_directory("logs/rdarts")
all_tensors

{'lora_darts_s2_9003.log': {'error': None,
  'normal': (tensor([[0.5024, 0.4976],
           [0.4880, 0.5120],
           [0.4583, 0.5417],
           [0.5233, 0.4767],
           [0.5462, 0.4538],
           [0.4937, 0.5063],
           [0.3447, 0.6553],
           [0.6089, 0.3911],
           [0.4724, 0.5276],
           [0.4962, 0.5038],
           [0.4626, 0.5374],
           [0.4582, 0.5418],
           [0.4081, 0.5919],
           [0.3077, 0.6923]]),
   tensor([ 4., 10.])),
  'reduction': (tensor([[0.5010, 0.4990],
           [0.4913, 0.5087],
           [0.4497, 0.5503],
           [0.4910, 0.5090],
           [0.4987, 0.5012],
           [0.5126, 0.4874],
           [0.4634, 0.5366],
           [0.4788, 0.5212],
           [0.4716, 0.5284],
           [0.4462, 0.5538],
           [0.4846, 0.5154],
           [0.4585, 0.5415],
           [0.5237, 0.4763],
           [0.5381, 0.4619]]),
   tensor([ 4., 10.]))},
 'lora_darts_s2_9002.log': {'error': None,
  'normal': (tensor([[0.48

In [8]:
{k:v for k,v in all_tensors.items() if "_s2_" in k and "lora" in k}

{'lora_darts_s2_9003.log': {'error': None,
  'normal': (tensor([[0.5024, 0.4976],
           [0.4880, 0.5120],
           [0.4583, 0.5417],
           [0.5233, 0.4767],
           [0.5462, 0.4538],
           [0.4937, 0.5063],
           [0.3447, 0.6553],
           [0.6089, 0.3911],
           [0.4724, 0.5276],
           [0.4962, 0.5038],
           [0.4626, 0.5374],
           [0.4582, 0.5418],
           [0.4081, 0.5919],
           [0.3077, 0.6923]]),
   tensor([ 4., 10.])),
  'reduction': (tensor([[0.5010, 0.4990],
           [0.4913, 0.5087],
           [0.4497, 0.5503],
           [0.4910, 0.5090],
           [0.4987, 0.5012],
           [0.5126, 0.4874],
           [0.4634, 0.5366],
           [0.4788, 0.5212],
           [0.4716, 0.5284],
           [0.4462, 0.5538],
           [0.4846, 0.5154],
           [0.4585, 0.5415],
           [0.5237, 0.4763],
           [0.5381, 0.4619]]),
   tensor([ 4., 10.]))},
 'lora_darts_s2_9002.log': {'error': None,
  'normal': (tensor([[0.48

In [9]:
lora_darts_count = [4, 4, 8, 6, 7, 6]
darts_count = [9, 6, 12, 6, 12, 5]

In [6]:
np.mean(lora_darts_count), np.mean(darts_count)

(5.833333333333333, 8.333333333333334)